# Names in the Bible

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from build_data import *

ot_dict = read_cbol_dict('../data/dhfhl')
nt_dict = read_cbol_dict('../data/dgfhl')
ot_dict_strong = ot_dict[ot_dict.strong.astype(int) < 8675]
cbol_dict = pd.concat([ot_dict_strong, nt_dict]).dropna().reset_index(drop=True)
cbol_dict['def_lines'] = cbol_dict.defs.str.split('\\\\r\\\\n')

Missing entry omitted at the moment.

In [3]:
ot_dict[ot_dict.defs.isna()]

,strong,defs
1144,07427,<NA>


In [4]:
ntna = nt_dict[nt_dict.defs.isna()]
ntna[ntna.strong.astype(np.int32) < 5625]

,strong,defs
1109,04236,<NA>
4157,04191,<NA>


## Add POS

In [5]:
cbol_dict['pos'] = cbol_dict.def_lines.transform(find_pos)
cbol_dict['jieba_pos'] = cbol_dict.defs.transform(jieba_pos)
cbol_dict.head()

,strong,defs,def_lines,pos,jieba_pos
0,06359,06359 patiyr {paw-teer'}\r\n\r\n源自 06362; TWOT...,"[06359 patiyr {paw-teer'}, , 源自 06362; TWOT - ...",形容詞,a
1,04000,04000 mabown {maw-bone'}\r\n\r\n源自 0995; TWOT ...,"[04000 mabown {maw-bone'}, , 源自 0995; TWOT - 2...",實名詞（作名詞,nr
2,02453,02453 Chakmowniy {khak-mo-nee'}\r\n\r\n源自 0244...,"[02453 Chakmowniy {khak-mo-nee'}, , 源自 02449; ...",陽性專有名詞,nr
3,03143,03143 Yowshibyah {yo-shi-yaw'}\r\n\r\n源自 03427...,"[03143 Yowshibyah {yo-shi-yaw'}, , 源自 03427 和 ...",陽性專有名詞,nr
4,03412,03412 Yarmuwth {yar-mooth'}\r\n\r\n源自 07311; 專...,"[03412 Yarmuwth {yar-mooth'}, , 源自 07311; 專有名詞...",專有名詞,ns


## Extra vocabs

In [6]:
name_dict = create_name_dict(cbol_dict)
name_dict

,strong,name,1,pos,jieba_pos
2,02453,哈摩尼或哈革摩尼,"""智慧""",陽性專有名詞,nr
3,03143,約示比,"""耶和華使我安居""",陽性專有名詞,nr
4,03412,耶末,"""高地""",專有名詞,ns
10,04722,瑪吉希錄,"""集會之處""",陰性複數專有名詞,ns
16,06126,阿谷或亞谷,"""險惡的""",陽性專有名詞,nr
...,...,...,...,...,...
14482,04099,彼西底,"""漆黑的""",專有名詞,ns
14489,05376,腓力,"""馬的愛好者""",陽性專有名詞,nr
14490,00490,安提阿,"'與之抗衡""",專有地名,ns
14492,02264,希律,"""英勇的""",陽性專有名詞,nz


In [7]:
def is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
        (cp >= 0x4E00 and cp <= 0x9FFF)
        or (cp >= 0x3400 and cp <= 0x4DBF)  #
        or (cp >= 0x20000 and cp <= 0x2A6DF)  #
        or (cp >= 0x2A700 and cp <= 0x2B73F)  #
        or (cp >= 0x2B740 and cp <= 0x2B81F)  #
        or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
        or (cp >= 0xF900 and cp <= 0xFAFF)
        or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False


name_dict['name_list'] = name_dict.name.str.split('[或，,、/]')
non_chinese = name_dict.name_list.explode().apply(lambda name: [char for char in name if not is_chinese_char(ord(char))]).explode().drop_duplicates()
non_chinese

2      NaN
211      ・
Name: name_list, dtype: object

## Generate names file

In [8]:
from hanziconv import HanziConv

def save_name_list(name_list: pd.DataFrame):
     names = name_list.explode('name_list')
     names['name_list'] = names.name_list.str.strip('黨族人的')\
                               .str.replace('・', '')\
                               .str.replace('毗', '毘')\
                               .apply(HanziConv.toSimplified)
     names[['name_list', 'jieba_pos']].drop_duplicates('name_list')\
                                      .to_csv('word_tokens/names.txt', index=False, header=None, sep=' ')

save_name_list(name_dict[['name_list', 'jieba_pos']])

## Check name occurrances

In [9]:
def search_in_bible(name_list):
    search_pattern = name_list.apply(lambda list: f'(?:{"|".join(list)})')\
                              .str.replace('[黨族人的]', '')\
                              .str.replace('・', '')\
                              .str.replace('毗', '毘')
    dots = r'[•‧．・\-]'
    unv = read_bible('../data/dnstrunv')
    unv_text_no_dots = unv.text.str.replace(dots, '')
    name_count = search_pattern.apply(lambda pat: unv_text_no_dots.str.count(pat).sum())
    return name_count

In [10]:
name_dict['occurrance'] = search_in_bible(name_dict.name_list)
name_dict.sort_values('occurrance', ascending=False)[:20]

,strong,name,1,pos,jieba_pos,name_list,occurrance
6010,03864,,"""受苦的"" (#耶46:9|)",專有名詞,nz,[],1096104
4353,03072,耶和華,是我們的公義,陽性專有名詞,nz,[耶和華],6980
7838,03068,耶和華,"""自有永有的""",專有名詞,nz,[耶和華],6980
7177,00237,以色,"""離開""",專有名詞,ns,[以色],2706
5072,03481,以色列人的,"見 以色列[3478] ""上帝勝過""",形容詞,nr,[以色列人的],2703
5207,03478,以色列,"""上帝勝過""",陽性專有名詞,nz,[以色列],2703
4976,03479,以色列,"""上帝勝過""",專有名詞,nr,[以色列],2703
5580,03482,以色列人的,"見 以色列(<3478>) ""上帝勝過""",陰性形容詞,nr,[以色列人的],2703
7138,00203,安,"""財富"" 或 ""精力""",陽性專有名詞,nr,[安],1751
14338,02424,耶穌,「耶和華是拯救」,陽性專有名詞,nr,[耶穌],1640


## Manual adjust names for UNV

In [11]:
name_dict[name_dict.occurrance == 0].to_csv('notexist.csv')

In [12]:
adjusted = pd.read_csv('unv_adjusted.csv', index_col=0)
adjusted['name_list'] = adjusted.name.str.split('[或，,、/]')
adjusted['occurrance'] = search_in_bible(adjusted.name_list)

In [13]:
adjusted[adjusted.occurrance == 0]

,strong,name,name_list,occurrance
1309,4723,古厄,[古厄],0
1755,5180,尼忽士但,[尼忽士但],0
5644,1045,伯・亞斯她錄,[伯・亞斯她錄],0
8043,1056,巴卡,[巴卡],0
8904,4515,拉丁文,[拉丁文],0
9164,3818,帕卡底亞那,[帕卡底亞那],0
9998,3496,尼亞波利,[尼亞波利],0
10598,5175,特羅基利翁,[特羅基利翁],0
11324,4513,拉丁文,[拉丁文],0
12104,3505,尼錄,[尼錄],0


In [14]:
name_dict.loc[adjusted.index, 'name'] = adjusted.name
name_dict.loc[adjusted.index]

,strong,name,1,pos,jieba_pos,name_list,occurrance
214,08188,梭琳,"""大麥""",陽性專有名詞,nr,[梭林],0
1309,04723,古厄,NaN,陽性名詞,ns,[古厄],0
1489,08234,沙斐,"""美麗""",專有名詞,nz,[沙婓],0
1634,05179,尼護施她,"""黃銅""",陰性專有名詞,nr,[尼護施他],0
1668,03026,伊迦爾撒哈杜他,"""以此石堆為見證""",專有名詞,ns,[伊迦爾撒哈杜地],0
...,...,...,...,...,...,...,...
14232,00828,奧古斯都,"""可敬的""",陽性專有名詞,nz,[亞古士督],0
14272,01179,低加坡里,"""十座城""",專有地名,ns,[低加波利],0
14348,02542,凱撒利亞,"""切斷的""",專有地名,ns,[該撒利亞],0
14468,00697,亞略巴古,"""戰神之巔""",專有地名,ns,[亞略古巴],0


In [15]:
name_dict['name_list'] = name_dict.name.str.split('[或，,、/]')
save_name_list(name_dict)